In [1]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
np.finfo(np.float64).max = 1e10

In [2]:
url = 'https://github.com/LeDataSciFi/data/raw/main/Firm%20Year%20Datasets%20(Compustat)/firms2020.csv'
firms_df = pd.read_csv(url).drop_duplicates('tic')

bod_df = pd.read_csv('./Input_data/DirectorComp.csv')
ceo_df = pd.read_csv('./Input_data/CEOComp.csv')

In [3]:
ceo_df.describe()

,EXECDIR,OLD_DATAFMT_FLAG,INTERLOCK,REPRICE,EXECRANK,CO_PER_ROL,SALARY,BONUS,STOCK_AWARDS,OPTION_AWARDS,...,TDC2_PCT,RET_YRS,COMMENT,GVKEY,EXECID,YEAR,RSTKVYRS,REJOIN,RELEFT,SIC
count,26767.00000,26767.0,26767.0,26767.0,1478.000000,26767.000000,26767.000000,26767.000000,2.676700e+04,2.676700e+04,...,21068.000000,0.0,0.0,26767.000000,26767.000000,26767.000000,0.0,0.0,0.0,26767.000000
mean,0.30758,0.0,0.0,0.0,3.123816,41633.237793,687.588032,213.962473,2.642567e+03,9.437737e+02,...,138.128526,NaN,NaN,45484.401464,36905.725333,2014.560690,NaN,NaN,NaN,4817.810027
std,0.46150,0.0,0.0,0.0,1.514619,13458.286879,476.515060,949.262743,1.291578e+04,1.425832e+04,...,7214.778424,NaN,NaN,61032.150088,13000.437730,2.870718,NaN,NaN,NaN,1846.527444
min,0.00000,0.0,0.0,0.0,1.000000,79.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,...,-100.000000,NaN,NaN,1045.000000,13.000000,2010.000000,NaN,NaN,NaN,100.000000
25%,0.00000,0.0,0.0,0.0,2.000000,32990.500000,448.120500,0.000000,5.562355e+02,0.000000e+00,...,-12.724500,NaN,NaN,6266.000000,28656.000000,2012.000000,NaN,NaN,NaN,3572.000000
50%,0.00000,0.0,0.0,0.0,3.000000,43902.000000,598.712000,0.000000,1.320151e+03,2.152320e+02,...,10.989500,NaN,NaN,12138.000000,39250.000000,2015.000000,NaN,NaN,NaN,4911.000000
75%,1.00000,0.0,0.0,0.0,4.000000,50768.000000,834.624500,0.000000,3.009438e+03,8.265285e+02,...,54.584250,NaN,NaN,63080.000000,46236.500000,2017.000000,NaN,NaN,NaN,6324.000000
max,1.00000,0.0,0.0,0.0,10.000000,68553.000000,18000.000000,32000.000000,1.927511e+06,2.283989e+06,...,913403.089000,NaN,NaN,316056.000000,62363.000000,2019.000000,NaN,NaN,NaN,9997.000000


In [4]:
ceo_df.dropna(axis=1, how='all', inplace=True)
# Calculate the total compensation for the CEO in that year
ceo_df = ceo_df[ceo_df['CEOANN'].notnull()]
ceo_df['TDC1'].describe() #only difference between TDC1 and TDC2 is the options preferences. TDC1 is bk value of options (Black- Scholes equation)

count      4811.000000
mean      11467.801272
std       10203.831261
min           0.001000
25%        6158.732500
50%        9658.115000
75%       14350.184000
max      280621.551000
Name: TDC1, dtype: float64

In [5]:
# ceo_df = ceo_df.drop(['CFOANN', 'PENSION_CHG', 'TOTAL_SEC', 'CHG_CTRL_PYMT', 'TOTAL_SEC_PCT', 'REPRICE', 'EXECRANK'], axis=1)
######trimmed this down to make it easier for regressions. We can look back if I missed any. 
ceo_df = ceo_df[['GVKEY', 'TICKER', 'CEOANN', 'SALARY', 'BONUS', 'STOCK_AWARDS', 'OPTION_AWARDS',
                 'OTHCOMP', 'TOTAL_CURR', 'AGE', 'TDC1', 'YEAR', 'BECAMECEO', 'JOINED_CO',
                 'LEFTOFC', 'GENDER', 'CUSIP', 'NAICSDESC', 'SIC']] 

In [6]:
bod_df = bod_df[['GVKEY', 'TICKER', 'TOTAL_SEC', 'OTHCOMP', 'NONEQ_INCENT', 'CASH_FEES', 'STOCK_AWARDS', 'OPTION_AWARDS',
                 'SPCODE', 'CUSIP', 'CONAME', 'YEAR']]

In [7]:
grouped_df = bod_df.groupby(['TICKER', 'YEAR'])[['TOTAL_SEC', 'OTHCOMP', 
                                                 'NONEQ_INCENT', 'CASH_FEES', 
                                                 'STOCK_AWARDS', 'OPTION_AWARDS']].sum().reset_index()

bod_df = bod_df.merge(grouped_df, on='TICKER', how='left')

bod_df = bod_df.rename(columns={'YEAR_x': 'YEAR'})
bod_df.drop(['YEAR_y'], axis=1)

bod_df = bod_df.rename(columns={'TOTAL_SEC_x': 'TOTAL_SEC'})
bod_df = bod_df.rename(columns={'TOTAL_SEC_y': 'total_director_comp'})

bod_df = bod_df.rename(columns={'OTHCOMP_x': 'OTHCOMP'})
bod_df = bod_df.rename(columns={'OTHCOMP_y': 'total_OTHCOMP'})

bod_df = bod_df.rename(columns={'NONEQ_INCENT_x': 'NONEQ_INCENT'})
bod_df = bod_df.rename(columns={'NONEQ_INCENT_y': 'total_NONEQ_INCENT'})

bod_df = bod_df.rename(columns={'CASH_FEES_x': 'CASH_FEES'})
bod_df = bod_df.rename(columns={'CASH_FEES_y': 'total_CASH_FEES'})

bod_df = bod_df.rename(columns={'STOCK_AWARDS_x': 'STOCK_AWARDS'})
bod_df = bod_df.rename(columns={'STOCK_AWARDS_y': 'total_STOCK_AWARDS'})

bod_df = bod_df.rename(columns={'OPTION_AWARDS_x': 'OPTION_AWARDS'})
bod_df = bod_df.rename(columns={'OPTION_AWARDS_y': 'total_OPTION_AWARDS'})

In [8]:
# bod_df.info()

In [9]:
bod_df = bod_df.drop_duplicates(subset=['TICKER', 'YEAR'], keep='first')

bod_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4792 entries, 0 to 486082
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   GVKEY                4792 non-null   int64  
 1   TICKER               4792 non-null   object 
 2   TOTAL_SEC            4792 non-null   float64
 3   OTHCOMP              4792 non-null   float64
 4   NONEQ_INCENT         4792 non-null   float64
 5   CASH_FEES            4792 non-null   float64
 6   STOCK_AWARDS         4792 non-null   float64
 7   OPTION_AWARDS        4792 non-null   float64
 8   SPCODE               4792 non-null   object 
 9   CUSIP                4792 non-null   object 
 10  CONAME               4792 non-null   object 
 11  YEAR                 4792 non-null   int64  
 12  YEAR_y               4792 non-null   int64  
 13  total_director_comp  4792 non-null   float64
 14  total_OTHCOMP        4792 non-null   float64
 15  total_NONEQ_INCENT   4792 non-null  

In [10]:
firms_df = pd.read_csv('./Input_data/Firm_data.csv')

In [11]:
firms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5767 entries, 0 to 5766
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gvkey     5767 non-null   int64  
 1   datadate  5767 non-null   object 
 2   fyear     5762 non-null   float64
 3   indfmt    5767 non-null   object 
 4   consol    5767 non-null   object 
 5   popsrc    5767 non-null   object 
 6   datafmt   5767 non-null   object 
 7   tic       5767 non-null   object 
 8   cusip     5767 non-null   object 
 9   conm      5767 non-null   object 
 10  curcd     5762 non-null   object 
 11  fyr       5762 non-null   float64
 12  act       4153 non-null   float64
 13  at        5739 non-null   float64
 14  bkvlps    4812 non-null   float64
 15  capx      4849 non-null   float64
 16  ceql      4858 non-null   float64
 17  csho      4813 non-null   float64
 18  dltt      5720 non-null   float64
 19  dt        4334 non-null   float64
 20  emp       5687 non-null   floa

In [12]:
firms_df['mkvalt'].describe()

count    5.078000e+03
mean     3.714428e+04
std      6.996852e+04
min      6.289170e+01
25%      8.459879e+03
50%      1.592204e+04
75%      3.356990e+04
max      1.073391e+06
Name: mkvalt, dtype: float64

In [13]:
# Tiny: companies with a market cap of less than $2 billion
# Medium: companies with a market cap between $2 billion and $10 billion
# Big: companies with a market cap between $10 billion and $100 billion
# Huge: companies with a market cap greater than $100 billion

In [14]:
# define the market capitalization categories
categories = {
    'Tiny': (0, 2_000),
    'Medium': (2_000, 10_000),
    'Big': (10_000, 100_000),
    'Huge': (100_000, float('inf'))
}

# create a new column with the market capitalization category for each company
firms_df['size_category'] = pd.cut(firms_df['mkvalt'], bins=[categories[cat][0] for cat in categories.keys()] + [float('inf')], labels=categories.keys())

In [15]:
take = firms_df[['tic', 'fyear', 'mkvalt', 'size_category']]
take = take.rename(columns={'tic': 'TICKER'})
take = take.rename(columns={'fyear': 'YEAR'})
take = take.rename(columns={'mkvalt': 'MKVALT'})
take = take.rename(columns={'size_category': 'SIZE_CATEGORY'})


In [16]:
# ceo_df = ceo_df.drop(ceo_df.columns[18], axis=1)
take.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5767 entries, 0 to 5766
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   TICKER         5767 non-null   object  
 1   YEAR           5762 non-null   float64 
 2   MKVALT         5078 non-null   float64 
 3   SIZE_CATEGORY  5078 non-null   category
dtypes: category(1), float64(2), object(1)
memory usage: 141.1+ KB


In [17]:
import pandas as pd
import statsmodels.formula.api as smf

ceo_df = ceo_df.merge(take, on = ['TICKER', 'YEAR'], how = 'inner')
bod_df = bod_df.merge(take, on = ['TICKER', 'YEAR'], how = 'inner')

ceo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5649 entries, 0 to 5648
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   GVKEY          5649 non-null   int64   
 1   TICKER         5649 non-null   object  
 2   CEOANN         5649 non-null   object  
 3   SALARY         5649 non-null   float64 
 4   BONUS          5649 non-null   float64 
 5   STOCK_AWARDS   5649 non-null   float64 
 6   OPTION_AWARDS  5649 non-null   float64 
 7   OTHCOMP        5649 non-null   float64 
 8   TOTAL_CURR     5649 non-null   float64 
 9   AGE            5649 non-null   float64 
 10  TDC1           5649 non-null   float64 
 11  YEAR           5649 non-null   int64   
 12  BECAMECEO      5613 non-null   object  
 13  JOINED_CO      1493 non-null   object  
 14  LEFTOFC        3530 non-null   object  
 15  GENDER         5649 non-null   object  
 16  CUSIP          5649 non-null   object  
 17  NAICSDESC      5649 non-null   ob

In [18]:
import hashlib
get_signature1 = lambda row_idx: hashlib.md5(str(ceo_df.iloc[row_idx]).encode('utf-8')).hexdigest()
get_signature2 = lambda row_idx: hashlib.md5(str(bod_df.iloc[row_idx]).encode('utf-8')).hexdigest()

# apply the lambda function to each row of the DataFrame to create a new column of signature indices
ceo_df['signature_index'] = ceo_df.index.to_series().apply(get_signature1)
bod_df['signature_index'] = bod_df.index.to_series().apply(get_signature2)

In [19]:
categories = {
    'Tiny': (0, 300),
    'Medium': (300, 10_000),
    'Big': (10_000, 200_000),
    'Huge': (200_000, float('inf'))
}

# create a new column with the market capitalization category for each company
ceo_df['SIZE_CATEGORY'] = pd.cut(ceo_df['MKVALT'], bins=[categories[cat][0] for cat in categories.keys()] + [float('inf')], labels=categories.keys())
bod_df['SIZE_CATEGORY'] = pd.cut(bod_df['MKVALT'], bins=[categories[cat][0] for cat in categories.keys()] + [float('inf')], labels=categories.keys())

In [20]:
ceo_df.to_csv('./input_data_clean/CEO_DF.csv')
bod_df.to_csv('./input_data_clean/BOD_DF.csv')
firms_df.to_csv('./input_data_clean/FIRMS.csv')

In [21]:
ceo_df['SIZE_CATEGORY'].describe()

count     5028
unique       4
top        Big
freq      3058
Name: SIZE_CATEGORY, dtype: object

In [6]:
import pandas as pd

ceo_df = pd.read_csv('./input_data_clean/CEO_DF.csv')
bod_df = pd.read_csv('./input_data_clean/BOD_DF.csv')

In [7]:
categories = {
    'Small': (0, 10_000),
    'Medium': (10_000, 200_000),
    'Large': (200_000, float('inf'))
}

# create a new column with the market capitalization category for each company
ceo_df['size_category'] = pd.cut(ceo_df['mkvalt'], bins=[categories[cat][0] for cat in categories.keys()] + [float('inf')], labels=categories.keys())
bod_df['size_category'] = pd.cut(bod_df['mkvalt'], bins=[categories[cat][0] for cat in categories.keys()] + [float('inf')], labels=categories.keys())

In [8]:
ceo_df.to_csv('./input_data_clean/CEO_DF.csv')
bod_df.to_csv('./input_data_clean/BOD_DF.csv')

In [21]:
import pandas as pd

ceo_df = pd.read_csv('./input_data_clean/CEO_DF.csv')
bod_df = pd.read_csv('./input_data_clean/BOD_DF.csv')

In [22]:

ceo_df = ceo_df[['signature_index', 'year','size_category', 'tdc1', 'age','gender','mkvalt','ownershippower','yearserved','prestiegepower','ownershipratio']]
ceo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115859 entries, 0 to 115858
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   signature_index  115859 non-null  object 
 1   year             115859 non-null  int64  
 2   size_category    80238 non-null   object 
 3   tdc1             115859 non-null  float64
 4   age              115859 non-null  float64
 5   gender           115859 non-null  object 
 6   mkvalt           80238 non-null   float64
 7   ownershippower   115859 non-null  int64  
 8   yearserved       115424 non-null  float64
 9   prestiegepower   115859 non-null  int64  
 10  ownershipratio   115859 non-null  float64
dtypes: float64(5), int64(3), object(3)
memory usage: 9.7+ MB


In [12]:
bod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8773 entries, 0 to 8772
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0.3         8773 non-null   int64  
 1   Unnamed: 0.2         8773 non-null   int64  
 2   Unnamed: 0.1         8773 non-null   int64  
 3   Unnamed: 0           8773 non-null   int64  
 4   gvkey                8773 non-null   int64  
 5   ticker               8773 non-null   object 
 6   total_sec            8773 non-null   float64
 7   othcomp              8773 non-null   float64
 8   noneq_incent         8773 non-null   float64
 9   cash_fees            8773 non-null   float64
 10  stock_awards         8773 non-null   float64
 11  option_awards        8773 non-null   float64
 12  spcode               8773 non-null   object 
 13  cusip                8773 non-null   object 
 14  coname               8773 non-null   object 
 15  year                 8773 non-null   i

In [23]:
bod_df = bod_df[['total_director_comp', 'year','mkvalt','size_category','signature_index','net_income','num_employees','debt_to_equity','assets_in_place','capex_by_assets','return_on_equity']]

In [24]:
ceo_df.to_csv('./input_data_clean/romoCEO_DF.csv')
bod_df.to_csv('./input_data_clean/romoBOD_DF.csv')